In [62]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [63]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv

In [132]:


client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages = [
    {"role": "system", "content": "You are a weather assistant. Help the user to determine the weather at a specific location."},
    {"role":"user","content": "whats the weather in pittsburgh rn"}
    ],
)
print(response.choices[0].message.content)

I'm unable to provide real-time weather information as my training only includes data up to October 2023. However, you can check a reliable weather website or app for the current weather in Pittsburgh.


In [65]:
def get_current_weather(location):
    # Simulate a weather API response
    weather_info = {
        "location": location,
        "temperature": 72,
        "forecast": "Partly cloudy with a chance of rain."
    }
    return weather_info

In [66]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "use this function to get the status of the current weather, this would be useful when the user asks 'what is the weather right now?' or when the user asks 'should I walk to Tepper building?' for example",
            "parameters": {
                "type": "object",
                "properties": {
                    "location":{
                        "type" : "string",
                        "description": "The location you want the weather for"
                    },
                },
            },
            "required": ["location"],
            "additionalProperties": False,
        }
    }
]

In [93]:
messages = [
    {"role": "system", "content": "You are a weather assistant. Help the user to determine the weather at a specific location."},
    #{"role":"user","content": "Tell me about CMU Scotty Labs"},
    
    {"role":"user","content": "whats the weather in vietnam rn"}
]

# Function Calling Walk Through

In [133]:
messages = [
    {"role": "system", "content": "You are a weather assistant. Help the user to determine the weather at a specific location."},
    #{"role":"user","content": "Tell me about CMU Scotty Labs"},
    
    {"role":"user","content": "whats the weather in vietnam rn"}
]

response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages = messages,
    tools = tools
)
messages.append(response.choices[0].message)
print(messages)
print(response)

[{'role': 'system', 'content': 'You are a weather assistant. Help the user to determine the weather at a specific location.'}, {'role': 'user', 'content': 'whats the weather in vietnam rn'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_wm5MdyGhPGy0O6MhJEa46Ggv', function=Function(arguments='{"location":"Vietnam"}', name='get_current_weather'), type='function')], refusal=None)]
ChatCompletion(id='chatcmpl-ACZWqRV9XOSMcdqLQhSCJrZUAhYl1', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_wm5MdyGhPGy0O6MhJEa46Ggv', function=Function(arguments='{"location":"Vietnam"}', name='get_current_weather'), type='function')], refusal=None))], created=1727559872, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_f85bea6784', usage=CompletionUsa

In [134]:
tool_call = response.choices[0].message.tool_calls
flag = False
if tool_call:
    print("Function Called")
    print(tool_call)
    flag = True
else:
    print("Function Not Called")
    print(response.choices[0].message.content)
    print("\n\n\n")

Function Called
[ChatCompletionMessageToolCall(id='call_wm5MdyGhPGy0O6MhJEa46Ggv', function=Function(arguments='{"location":"Vietnam"}', name='get_current_weather'), type='function')]


In [135]:
#weather_update = {'location': 'Vietnam', 'temperature': 72, 'forecast': 'Partly cloudy with a chance of rain.'}
if flag:
    tool_call = response.choices[0].message.tool_calls[0]

    arguments = json.loads(tool_call.function.arguments)
    location = arguments.get("location")
    weather_update = get_current_weather(location)
    print(f"location recognised bylChatgpt: {location}") 
    print(f"weather update: {weather_update}")

    function_call_result_message = {
        "role": "tool",
        "content": json.dumps({
            "location": weather_update['location'],
            "forecast": weather_update['forecast'],
            "temperature": weather_update['temperature'],
        }),
        "tool_call_id": response.choices[0].message.tool_calls[0].id
    }


location recognised bylChatgpt: Vietnam
weather update: {'location': 'Vietnam', 'temperature': 72, 'forecast': 'Partly cloudy with a chance of rain.'}


In [136]:
messages.append(function_call_result_message)
completion_payload = {
    "model": "gpt-4o-mini",
    "messages": messages
}

In [137]:
response = client.chat.completions.create(
    model=completion_payload["model"],
    messages=completion_payload["messages"]
)

print(response.choices[0].message.content)

The current weather in Vietnam is partly cloudy with a chance of rain. The temperature is around 72°F.
